# Halley's method with Tensorflow

Halley's method is a root-finding algorithm that can be used in one real variable functions with continuous second derivative. This notebook presents the implementation of Halley's method to find a root for a fourth degree polinomial.

Halley's method uses a initial value of X and iterates through the following formula:

\begin{equation*}
X_{n+1} = Xn - \frac{2f(x_n)*f'(x_n)}{2(f'(x_n))^2 - 2f(x_n)f''(x_n)}
\end{equation*}


This notebook uses a maximum number of iterations and stop criteria that consists of the difference between the value of $X_n$ and $X_{n+1}$ to determ the root of the polynomial.

In [1]:
#Importing library and enabling eager execution
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.enable_eager_execution()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
def root(i, a, x, x0):
    
    #Implementing automatic differentiation (second derivative)
    with tf.GradientTape() as t:
        t.watch(x)
        
        #Implementing automatic differentiation (first derivative)
        with tf.GradientTape() as t2:
            t2.watch(x)
            
            #fourth degree polinomial
            f_x = a[0] + a[1]*x + a[2]*tf.pow(x,2) + a[3]*tf.pow(x,3) + a[4]*tf.pow(x,4)
            f_x = tf.identity(f_x, name="f_x")
        
        #First derivative
        f_x1 = t2.gradient(f_x, x)
        f_x1 = tf.identity(f_x1, name="f_x1")
    
    #Second derivative
    f_x2 = t.gradient(f_x1, x)
    f_x2 = tf.identity(f_x2, name="f_x2")
    
    #Halley's method
    x0 = x
    x = x - (2*f_x*f_x1)/(2*tf.pow(f_x1,2) - f_x*f_x2)
    x = tf.identity(x, name="x")

    return [i+1, a, x, x0]

In [3]:
#If the difference between x and x0 is less than stop end the tf.while_loop
def condition(i, a, x, x0):
    diff = tf.abs(x - x0)
    return tf.less(stop, diff)

In [4]:
#Iterator for tf.while_loop
i = tf.constant(0)

#The coefficients a0 to a4 
a = tf.constant([7.0, -3.0, 0.0, 5.0, 2.0])  

#Initial guess x
x = tf.constant(1.0)

#Previous value of x (needs a start value to be updated in the function root)
previous = tf.constant(10.0)

#Stop criteria
stop = 0.0001

#Calculating the root
output =  tf.while_loop(     
          condition, 
          root, 
          loop_vars=[i, a, x, previous],
          maximum_iterations=10000)

print("Result after {0} iterations is {1}.".format(output[0], output[2]))

Result after 10000 iterations is -1.1812458038330078.
